In [4]:
import pyodbc
import os
import pandas as pd
from zipfile import ZipFile

import numpy as np
from datetime import datetime as dt

from dateutil.relativedelta import relativedelta
from calendar import month_name, different_locale

# mdb_to_df

In [5]:
def mdb_to_df(file_name, sql):

    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        fr'DBQ={file_name};'
    )
    print(conn_str)
    cnxn = pyodbc.connect(conn_str)

    df = pd.read_sql(sql, cnxn)

    print(f'{file_name} Loaded')
    return df

# Read SUM

In [6]:
def read_sum(period):
    usecols_sum = """
    SELECT CDbl(TimeOn) AS TOn, CDbl(TimeOff) AS TOff,
    StationNr, Alarmcode, ID, Parameter
    FROM tblAlarmLog WHERE TimeOff IS NOT NULL
    union
    SELECT CDbl(TimeOn) AS TOn, TimeOff AS TOff,
    StationNr, Alarmcode, ID, Parameter
    FROM tblAlarmLog WHERE TimeOff IS NULL
    """
    
    file_name = f'../DATA/SUM/{period}-sum.mdb'

    alarms = mdb_to_df(file_name=file_name, sql=usecols_sum)

    alarms['TOn'] = sqldate_to_datetime(alarms['TOn'])
    alarms['TOff'] = sqldate_to_datetime(alarms['TOff'])

    alarms.rename(columns={'TOn': 'TimeOn',
                           'TOff': 'TimeOff'}, inplace=True)

    alarms = alarms[alarms.StationNr >= 2307405]

    alarms = alarms[
        alarms.StationNr <= 2307535].reset_index(
        drop=True)

    alarms.dropna(subset=['Alarmcode'], inplace=True)

    alarms.reset_index(drop=True, inplace=True)

    alarms.Alarmcode = alarms.Alarmcode.astype(int)

    return alarms


def sqldate_to_datetime(column):
    
    try:

        column = column.str.replace(',', '.').astype(float)
    except:
        pass

    day_parts = np.modf(column.loc[~column.isna()])

    column = column.fillna(pd.NaT)
    
    column.loc[column.notna()] = (
        dt(1899, 12, 30) +
        day_parts[1].astype('timedelta64[D]', errors='ignore') +
        (day_parts[0] * 86400000).astype('timedelta64[ms]', errors='ignore')
    )

    column = column.fillna(pd.NaT)
    
    
    return column

# Cascade

In [40]:
# Determine alarms real periods
def cascade(df):

    df.reset_index(inplace=True, drop=True)
    df['TimeOffMax'] = df.TimeOff.cummax().shift()

    df.at[0, 'TimeOffMax'] = df.at[0, 'TimeOn']

    return df


# looping through turbines and applying cascade method
def apply_cascade(result_sum):

    # Sort by alarm ID
    result_sum.sort_values(['ID'], inplace=True)
    df = result_sum.groupby('StationId').apply(cascade)

    mask_root = (df.TimeOn.values >= df.TimeOffMax.values)
    mask_children = (df.TimeOn.values < df.TimeOffMax.values) & (
        df.TimeOff.values > df.TimeOffMax.values)
    mask_embedded = (df.TimeOff.values <= df.TimeOffMax.values)

    df.loc[mask_root, 'NewTimeOn'] = df.loc[mask_root, 'TimeOn']
    df.loc[mask_children, 'NewTimeOn'] = df.loc[mask_children, 'TimeOffMax']
    df.loc[mask_embedded, 'NewTimeOn'] = df.loc[mask_embedded, 'TimeOff']

    df.drop(columns=['TimeOffMax'], inplace=True)

    df.reset_index(inplace=True, drop=True)

    TimeOff = df.TimeOff
    NewTimeOn = df.NewTimeOn

    df['RealPeriod'] = abs(TimeOff - NewTimeOn)

    mask_siemens = (df['Error Type'] == 1)
    mask_tarec = (df['Error Type'] == 0)

    df['Period Siemens(s)'] = df[mask_siemens].RealPeriod  # .dt.seconds
    df['Period Tarec(s)'] = df[mask_tarec].RealPeriod  # .dt.seconds
    # df['RealPeriod'] = df['RealPeriod'].dt.seconds

    return df

# Periods

In [41]:
def get_month_name(month_no, locale):
    with different_locale(locale):
        return month_name[month_no].capitalize()
    

period = input('period ex: "2020-07"') or '2020-07'

period_dt = dt.strptime(period, "%Y-%m")
period_month = period_dt.month
period_month_name = get_month_name(period_month, 'french')
period_year = period_dt.year

days_in_period = pd.Period(f'{period}').days_in_month

previous_period_dt = period_dt + relativedelta(months=-1)
previous_period = previous_period_dt.strftime("%Y-%m")

lastday_period_dt = period_dt + relativedelta(months=1, days=-1)
lastday_period = lastday_period_dt.strftime("%Y-%m-%d")


lastday_previous_period_dt = period_dt + relativedelta(days=-1)
lastday_previous_period = lastday_previous_period_dt.strftime("%Y-%m-%d")

days_to_previous_period = len(pd.date_range(
    f'{period_year}-01-01', lastday_previous_period_dt))

# Import last period cumul

In [9]:
df_last_ax1 = pd.read_excel(f'./input/{previous_period}/output_xlsxwriter.xlsx', sheet_name= 'ax1', index_col=0)

last_boost_cumul = df_last_ax1.iat[0, 7]
last_ratio_cumul = df_last_ax1.iat[0, 8]
last_mtbf_cumul = df_last_ax1.iat[0, 9]
last_mttr_cumul = df_last_ax1.iat[0, 10]
last_mtti_cumul = df_last_ax1.iat[0, 11]
last_onee_cumul = df_last_ax1.iat[0, 12]

In [10]:
df_last_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0,4.28,0.68,3.60,0,0,5087.88,1713.575587,99.793401,23.806976,8.847338,7.084972,242735.361
Indicateurs du mois 2021-03 :,0,4.66,0.84,3.82,0,0,1846.59,686.092594,99.770109,23.734153,10.523184,9.225580,90866.556


# ONEE

In [38]:
onee_tah = pd.read_table(f'./input/{period}/CCH TAREC TR NORM {str(period_month).zfill(2)} {period_year}.XLS')
onee_nord = pd.read_table(f'./input/{period}/CCH TAREC NORD NORM {str(period_month).zfill(2)} {period_year}.XLS')

onee = onee_tah.append(onee_nord)

onee_received = onee['kWh-Rec'].sum()/6/1000

In [ ]:
onee_tah = pd.read_excel(
    f'./input/{period}/CCH TAREC TR NORM {str(period_month).zfill(2)} {period_year}.xlsx')
onee_nord = pd.read_excel(
    f'./input/{period}/CCH TAREC NORD NORM {str(period_month).zfill(2)} {period_year}.xlsx')

onee = onee_tah.append(onee_nord)

onee_received = onee['kWh-Rec'].sum()/6/1000


In [39]:
onee.groupby(['Date', 'Time']).sum()['kWh-Del'].max()


1518

In [21]:
onee

,Unnamed: 0,Account:,Date,Time,Int.Len,kWh-Del,kVARh-Del,kWh-Rec,kVARh-Rec
0,TR,NORM,01/02/2021,00:10,10,0,11220,59400,0
1,TR,NORM,01/02/2021,00:20,10,0,12144,51216,0
2,TR,NORM,01/02/2021,00:30,10,0,12936,63624,0
3,TR,NORM,01/02/2021,00:40,10,0,13596,73656,0
4,TR,NORM,01/02/2021,00:50,10,0,13464,65868,0
...,...,...,...,...,...,...,...,...,...
4027,NORD,NORM,28/02/2021,23:20,10,0,9900,17820,0
4028,NORD,NORM,28/02/2021,23:30,10,0,9240,15510,0
4029,NORD,NORM,28/02/2021,23:40,10,0,7260,12540,0
4030,NORD,NORM,28/02/2021,23:50,10,0,6930,10890,0


In [9]:
onee_received

74786.624

In [9]:
onee_period = float(input('onee_period') or onee_received)

if period_month == 1:
    onee_cumul = onee_period
else:
    onee_cumul = (last_onee_cumul + onee_period)

In [10]:
last_onee_cumul

242735.361

# df ax 19 Performance ratio 

In [12]:

# df_ax19 = pd.read_html(f'./input/{period}/TAREC REPORTING.html', thousands=' ', decimal=',')[0]

# df_ax19['Turbine'] = df_ax19['Turbine'].str.extract(r'\[[^\d]*(\d+)[^\d]*\]').values.astype(int)

# df_ax19 = df_ax19[['Turbine', 'Ratio']]

# df_ax19.Ratio.mean()

# # last_ratio_cumul = input('last_ performance_ratio_cumul') or 94.846

# ratio_period = df_ax19.Ratio.mean()

# ratio_cumul = (last_ratio_cumul*days_to_previous_period + ratio_period*days_in_period)/(days_to_previous_period+days_in_period)

# df_ax19

# Calcul mois

In [42]:
import pandas as pd

df = pd.read_csv(f'../Availability_Warranty_Dash/monthly_data/results/{period}-Availability.csv',
                    decimal=',', sep=';')
df.columns = ['_'.join(str(v) for v in tup) if type(tup) is tuple else tup for tup in df.columns]
df.TimeStamp = pd.to_datetime(df.TimeStamp)
df.to_pickle(f'../Data/results/{period}.pkl')

In [14]:
# import pandas as pd

# for i in range(1, 13):
    
#     period = f'2020-{str(i).zfill(2)}'
#     df = pd.read_csv(f'../Availability_Warranty_Dash/monthly_data/results/{period}-Availability.csv',
#                         decimal=',', sep=';')
#     df.columns = ['_'.join(str(v) for v in tup) if type(tup) is tuple else tup for tup in df.columns]
#     df.TimeStamp = pd.to_datetime(df.TimeStamp)
#     df.to_pickle(f'../Data/results/{period}.pkl')

In [11]:
results = pd.read_pickle(f'../DATA/results/{period}.pkl')
results['StationId'] = results['StationId'] - 2307404

In [12]:
alarms = read_sum(period)
alarms.rename(columns={'StationNr': 'StationId'}, inplace=True)
alarms['StationId'] = alarms['StationId'] - 2307404

DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-04-sum.mdb;
../DATA/SUM/2021-04-sum.mdb Loaded


In [18]:
results.head(2)

,Unnamed: 0,StationId,TimeStamp,RealPeriod,Period 0(s),Period 1(s),UK Text,Duration 2006(s),wtc_kWG1Tot_accum,wtc_kWG1TotE_accum,...,next_AcWindSp,EL_PowerRed,EL_2006,EL_wind,Duration lowind(s),EL_wind_start,Duration lowind_start(s),EL_alarm_start,Duration alarm_start(s),EL_indefini_left
0,0,1,2021-04-01 00:10:00,0.0,0.0,0.0,0,0.0,-2.0,0.0,...,0.37745,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2021-04-01 00:20:00,0.0,0.0,0.0,0,0.0,-2.0,0.0,...,0.58221,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# abbreviations
Frame = pd.DataFrame
c1 = 'Alarmcode'
c2 = 'Error Group'

error_group = pd.concat([Frame({c1: range(901, 2101), c2: 'System'}),
                         Frame({c1: range(2101, 3000), c2: 'Generator'}),
                         Frame({c1: range(3100, 4000), c2: 'Hub'}),
                         Frame({c1: range(4100, 5000), c2: 'Gear'}),
                         Frame({c1: range(5000, 6000), c2: 'Grid'}),
                         Frame({c1: range(6100, 7000), c2: 'Rotor'}),
                         Frame({c1: range(7100, 8000), c2: 'Hydraulics'}),
                         Frame({c1: range(8000, 8400), c2: 'Environement'}),
                         Frame({c1: range(8450, 9000),
                                c2: 'Turbine cond...'}),
                         Frame({c1: range(9100, 10000), c2: 'Brake'}),
                         Frame({c1: range(10100, 11000), c2: 'Yaw'}),
                         Frame({c1: range(11100, 12000), c2: 'PFC'}),
                         Frame({c1: range(12100, 13000), c2: 'Transformer'}),
                         Frame({c1: range(13000, 14000), c2: 'Converter-1'}),
                         Frame({c1: range(14000, 15000), c2: 'Gen.inverter'}),
                         Frame({c1: range(15000, 16000), c2: 'Grid inverter'}),
                         Frame({c1: range(16000, 17000), c2: 'Main bearing'}),
                         Frame({c1: range(17000, 18300), c2: 'Converter-2'}),
                         Frame({c1: range(62001, 64000), c2: 'Controller'}),
                         Frame({c1: range(64000, 65200), c2: 'MISCELLANEOUS'})])


In [14]:
reorder = ['System',
           'Generator',
           'Hub',
           'Gear',
           'Grid',
           'Rotor',
           'Hydraulics',
           'Environement',
           'Turbine cond...',
           'Brake',
           'Yaw',
           'PFC',
           'Transformer',
           'Converter-1',
           'Gen.inverter',
           'Grid inverter',
           'Main bearing',
           'Converter-2',
           'Controller',
           'MISCELLANEOUS']

In [15]:
error_list = pd.read_excel(
    r'Error_Type_List_Las_Update_151209.xlsx',
    usecols=lambda x: x != 'Type Selected')

error_list.Alarmcode = error_list.Alarmcode.astype(int)  # ,errors='ignore'

error_list.drop_duplicates(subset=['Alarmcode'], inplace=True)

error_list = error_list.merge(error_group, on='Alarmcode', how='left')

# ------------------------------------------------------------------------
''' label scada alarms with coresponding error type
and only keep alarm codes in error list'''
result_sum = pd.merge(alarms, error_list,
                        on='Alarmcode',
                        how='inner', sort=False)

# Remove warnings
result_sum = result_sum.loc[result_sum['Error Type'].isin([1, 0])]

# apply cascade
alarms_result_sum = apply_cascade(result_sum)

# only keep  parent alarms
parent_result_sum = alarms_result_sum.query('TimeOn == NewTimeOn')

# dash duree
main_result_sum = alarms_result_sum.query('RealPeriod > @pd.Timedelta(0)')
# main_result_sum['ALL duree'] = main_result_sum['TimeOff'] -main_result_sum['TimeOn']

In [16]:
alarms

,TimeOn,TimeOff,StationId,Alarmcode,ID,Parameter
0,2021-04-01 00:00:28.000,NaT,34,159,4321654,AccCh:V103/120 OK ...
1,2021-04-01 00:01:19.010,NaT,34,111,4321656,Blade A ...
2,2021-04-01 00:01:30.019,NaT,34,111,4321657,Blade B ...
3,2021-04-01 00:01:41.029,NaT,34,111,4321658,Blade C ...
4,2021-04-01 00:01:54.000,2021-04-01 00:05:22.999,97,3130,4321659,...
...,...,...,...,...,...,...
168111,2021-04-30 23:51:53.069,NaT,29,67,4514223,.0 ...
168112,2021-04-30 23:51:53.079,NaT,29,68,4514224,627 ...
168113,2021-04-30 23:51:53.090,NaT,29,69,4514225,17.0 ...
168114,2021-04-30 23:56:27.079,NaT,12,127,4514230,...


## Power Boost

In [17]:
def read_boost(file_name):
    usecols_cnt = '''TimeStamp, StationId, wtc_BoostKWh_endvalue'''

    sql_cnt = f"Select {usecols_cnt} FROM tblSCTurCount;"

    cnt = mdb_to_df(file_name=file_name, sql=sql_cnt)

    cnt['TimeStamp'] = pd.to_datetime(
        cnt['TimeStamp'], format='%m/%d/%y %H:%M:%S')

    return cnt

def boost_endvalue_check(df):
    df = df.reset_index().sort_values('TimeStamp')
    df.drop('index', axis=1, inplace=True)
    clmn = df['wtc_BoostKWh_endvalue']
    x = (clmn.iat[-1] - clmn.iat[0])/1000
    # df = df.diff().query('wtc_BoostKWh_endvalue < 0')
    if x < 0:
        return 0
    else:
        return x

In [18]:

# last_boost_cumul = input('last_boost_cumul') or 3458.1

boost = read_boost(f'../Availability_Warranty_Dash/monthly_data/uploads/{period}/{period}-cnt.mdb')

boost_period = boost.groupby('StationId').apply(lambda df: boost_endvalue_check(df)).sum()

if period_month == 1:
    boost_cumul = boost_period
else:
    boost_cumul = last_boost_cumul + boost_period

DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../Availability_Warranty_Dash/monthly_data/uploads/2021-04/2021-04-cnt.mdb;
../Availability_Warranty_Dash/monthly_data/uploads/2021-04/2021-04-cnt.mdb Loaded


In [19]:
boost_period, boost_cumul

(456.0676016845703, 2169.643188598632)

## MTBF MTTR 

In [20]:
manual_stop = [1001,
               1002,
               1005,
               1007,
               1008,
               1009,
               1015,
               1016,
               1017,
               1021,
               1025,
               1026,
               1027,
               8216,
               8217,
               8222,
               ]

In [21]:
query = '(Alarmcode not in @manual_stop) & (RealPeriod >= @pd.Timedelta("2H"))'

freq_period = len(main_result_sum.query(query))

duree_period = (main_result_sum.query(query)
                               .RealPeriod
                               .sum().total_seconds()
                               / 3600)

mtbf_period = (days_in_period * 24 * 131 - duree_period) / freq_period / 24

mtbf_period

43.550348134233246

In [22]:
mttr_period = duree_period/freq_period

mttr_period

14.5669256772784

In [23]:
# Generate the new calculated MTTBF and MTTR for each error group

MTBF_MTTR_cat = (main_result_sum.query(query).groupby('Error Group')
                                .agg(duree=('RealPeriod', lambda x: x.sum().total_seconds() / 3600),
                                     freq=('TimeOn', 'count'),
                                     )
                                .rename_axis('Catégorie'))

temps_total_period = days_in_period * 24 * 131

duree_period = MTBF_MTTR_cat['duree']
freq_period = MTBF_MTTR_cat['freq']


MTBF_MTTR_cat['MTTR en h'] = duree_period/freq_period

MTBF_MTTR_cat['MTBF en j'] = (
    temps_total_period - duree_period) / (freq_period * 24)


In [24]:
MTBF_MTTR_cat

,duree,freq,MTTR en h,MTBF en j
Catégorie,,,,
Brake,70.463333,7,10.066190,561.009147
Controller,176.647499,13,13.588269,301.741514
Converter-1,581.911945,14,41.565139,278.982405
Environement,76.186111,8,9.523264,490.853197
Gear,79.385555,16,4.961597,245.418267
Grid,3.186945,1,3.186945,3929.867211
Hub,79.852222,6,13.308704,654.445471
Hydraulics,77.174444,8,9.646805,490.848050
MISCELLANEOUS,14.612222,1,14.612222,3929.391157


## MTTI

In [25]:

def mtti(df, turbine):

    # df = main_result_sum.query('StationId == @turbine')
    df = (df.loc[(df.TimeOn == df.NewTimeOn).shift(-1).fillna(True)]
            .query('TimeOn != NewTimeOn').assign(root=False)
            .append(df.query('TimeOn == NewTimeOn').assign(root=True))
            .sort_values('ID'))

    df.loc[(df.root == False).shift(-1).fillna(False), 'TimeOff'] = df.loc[(df.root == False)]['TimeOff'].values
    df['RealPeriod'] = df['TimeOff'] - df['TimeOn']

    df.drop(df.loc[df.root == False].index, inplace=True)

    df2 = alarms.query('StationId == @turbine & Alarmcode == 50000')

    df = (df2.append(df).sort_values('TimeOn').reset_index(drop=True))

    df = df.query('Alarmcode not in @manual_stop') 

    df = df.loc[(df['Alarmcode']).shift() != (df['Alarmcode'])]

    df = cascade(df)

    mask_root = (df.TimeOn.values >= df.TimeOffMax.values)
    mask_children = (df.TimeOn.values < df.TimeOffMax.values) & (
        df.TimeOff.values > df.TimeOffMax.values)
    mask_embedded = (df.TimeOff.values <= df.TimeOffMax.values)

    df.loc[mask_root, 'NewTimeOn'] = df.loc[mask_root, 'TimeOn']
    df.loc[mask_children, 'NewTimeOn'] = df.loc[mask_children, 'TimeOffMax']
    df.loc[mask_embedded, 'NewTimeOn'] = df.loc[mask_embedded, 'TimeOff']

    
    df.drop(df.query('TimeOn == NewTimeOn & Alarmcode == 50000').index, inplace=True)

    mask = (df.Alarmcode == 50000).shift(-1, fill_value=False)

    df.loc[mask, 'TimeTI'] = df.loc[(df.Alarmcode == 50000), 'TimeOn'].values

    df.loc[mask, 'TimeTI'] = df.loc[mask, 'TimeTI'] - df.loc[mask, 'TimeOn']

    df['TimeTI'] = df['TimeTI'].astype('timedelta64[ns]')


    return df


In [26]:
mtti_df = main_result_sum.groupby('StationId').apply(lambda x: mtti(x, x.name)).reset_index(drop=True)
mtti_df

,TimeOn,TimeOff,StationId,Alarmcode,ID,Parameter,UK Text,Error Type,Error Group,NewTimeOn,RealPeriod,Period Siemens(s),Period Tarec(s),root,TimeOffMax,TimeTI
0,2021-04-01 23:25:33.000,2021-04-01 23:29:02.999,1,3130,4327102,...,Pitch lubrication,0,Hub,2021-04-01 23:25:33.000,0 days 00:03:29.999000,NaT,0 days 00:03:29.999000,True,2021-04-01 23:25:33.000,NaT
1,2021-04-01 23:29:29.000,2021-04-01 23:43:41.000,1,10105,4327116,...,"Stopped, untwisting cables",0,Yaw,2021-04-01 23:29:29.000,0 days 00:14:12,NaT,0 days 00:14:12,True,2021-04-01 23:29:02.999,NaT
2,2021-04-06 23:25:30.000,2021-04-06 23:29:29.999,1,3130,4352911,...,Pitch lubrication,0,Hub,2021-04-06 23:25:30.000,0 days 00:03:59.999000,NaT,0 days 00:03:59.999000,True,2021-04-01 23:43:41.000,NaT
3,2021-04-14 10:24:30.999,2021-04-14 10:27:39.000,1,8155,4396444,...,Pri.Windspeed Range error,1,Environement,2021-04-14 10:24:30.999,0 days 00:03:08.001000,0 days 00:03:08.001000,NaT,True,2021-04-06 23:29:29.999,NaT
4,2021-04-14 10:27:49.999,2021-04-14 10:29:41.000,1,7106,4396469,...,"Pitch pump. time too long,stop",1,Hydraulics,2021-04-14 10:27:49.999,0 days 00:01:51.001000,0 days 00:01:51.001000,NaT,True,2021-04-14 10:27:39.000,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,2021-04-21 23:54:38.000,2021-04-21 23:58:07.999,130,3130,4419359,...,Pitch lubrication,0,Hub,2021-04-21 23:54:38.000,0 days 00:03:29.999000,NaT,0 days 00:03:29.999000,True,2021-04-21 13:26:22.050,NaT
1038,2021-04-23 02:18:50.999,2021-04-23 03:32:56.999,130,10100,4424127,...,Yaw motor(s) superheated,1,Yaw,2021-04-23 02:18:50.999,0 days 01:14:06,0 days 00:36:37.001000,NaT,True,2021-04-21 23:58:07.999,NaT
1039,2021-04-26 23:55:18.000,2021-04-26 23:58:48.000,130,3130,4470424,...,Pitch lubrication,0,Hub,2021-04-26 23:55:18.000,0 days 00:03:30,NaT,0 days 00:03:30,True,2021-04-23 10:03:36.517,NaT
1040,2021-04-01 05:35:56.000,2021-04-01 05:39:26.000,131,3130,4322356,...,Pitch lubrication,0,Hub,2021-04-01 05:35:56.000,0 days 00:03:30,NaT,0 days 00:03:30,True,2021-04-01 05:35:56.000,NaT


In [27]:
mtti_period = (mtti_df['TimeTI'].sum() / len(mtti_df.query('Alarmcode == 50000'))).total_seconds()/3600

mtti_period

4.321691044444445

## Graphe 3 ax5

In [28]:
df_ax5 = (main_result_sum.groupby('Error Group')
                  .agg(Freq=('Alarmcode', 'count'),
                       Durée=('RealPeriod', lambda x: x.sum().total_seconds()/3600))
                  .reindex(reorder)
                  .dropna()
                  .reset_index()
                  )

# df_ax5.plot(kind='bar', x='Error Group', ax=ax5)
df_ax5

,Error Group,Freq,Durée
0,System,637.0,2066.207555
1,Generator,1.0,0.002223
2,Hub,783.0,133.414999
3,Gear,39.0,89.001944
4,Grid,4.0,3.693333
5,Rotor,1.0,0.001111
6,Hydraulics,278.0,120.496114
7,Environement,148.0,119.658889
8,Brake,161.0,79.336012
9,Yaw,451.0,253.531122


## Graphe 4 ax6

In [29]:
df_ax6 = (main_result_sum.groupby('Alarmcode')
                .agg(Freq=('Alarmcode', 'count'),
                     Durée=('RealPeriod',
                               lambda x: x.sum().total_seconds()/3600))
                .sort_values('Durée', ascending=False)
                .head(20)
                .reset_index()
                .sort_values('Durée', ascending=False))

# df_ax6.plot(kind='bar', x='Alarmcode', ax=ax6)
df_ax6

,Alarmcode,Freq,Durée
0,1001,358,1387.030302
1,13902,42,463.966389
2,1026,20,226.835556
3,1007,52,213.295832
4,1025,105,207.407776
5,10100,50,130.691109
6,8160,81,80.758333
7,9306,103,61.658787
8,63300,11,55.651944
9,13216,17,49.120833


## ax7

In [30]:
most_categories = (main_result_sum.groupby('Error Group')
                                  .agg({'RealPeriod': np.sum})
                                  .sort_values('RealPeriod', ascending=False)
                                  .index.values)

realperiod_sum = main_result_sum.RealPeriod.sum()
percentage_per_category = (main_result_sum.groupby('Error Group')
                                  .agg(percentage=('RealPeriod',
                                                   lambda x: 100 * x.sum() / realperiod_sum))
                                  .sort_values('percentage', ascending=False))


df_most_categories = main_result_sum.loc[main_result_sum['Error Group'].isin(most_categories)].copy()

df_most_categories['Error Group'] = df_most_categories['Error Group'].astype("category")

df_most_categories['Error Group'].cat.set_categories(most_categories, inplace=True)

df_most_categories.sort_values('Error Group', inplace=True)


# ------------------------------------------------------------------------------------------------
def extract_important_alarms(df):

    df = df.groupby('Alarmcode').agg({'RealPeriod': np.sum}).reset_index()
    
    df['percentage'] = 100 * df['RealPeriod'] / df['RealPeriod'].sum()
    df.sort_values('percentage', ascending=False, inplace=True)
    df['percentage roll'] = df['percentage'].cumsum()
    df.set_index('Alarmcode', inplace=True)


#     for i in range(1,100):
#         if len(df.query('`percentage roll` < @i')) < 3:
#             continue
#         else:
#             df = df.query('`percentage roll` < @i')
#         break

    return df

# ------------------------------------------------------------------------------------------------

df_ax7 = (df_most_categories.groupby('Error Group', sort=False)
                            .apply(lambda x: extract_important_alarms(x))
                            .reset_index()
                            .merge(error_list[['UK Text', 'Alarmcode']], how='left')
         )

# df_ax7 = (df_most_categories.groupby('Error Group')
#                              .apply(lambda x: (x.groupby('Alarmcode')
#                                                 .agg({'RealPeriod': np.sum})
#                                                 .sort_values('RealPeriod')
#                                                 .tail(4)))
#                              .reset_index()
#                              .merge(error_list[['UK Text', 'Alarmcode']]))

# ------------------------------------------------------------------------------------------------
ax7 = ''
for cat in most_categories:
    
    str1_ax7 = ''
    
    for i, row in df_ax7.query('`Error Group` == @cat').iterrows():
        
        Alarmcode = row['Alarmcode']
        UK_Text = row['UK Text']
        Percentage = round(row['percentage'], 1)
                       
        str1_ax7 += f'\n\t-\"{Alarmcode} - {UK_Text} - {Percentage}% \".'
        
    
    str2_ax7 = f"""
La catégorie {cat} - {round(percentage_per_category.loc[cat, 'percentage'], 2)}%, les alarmes les plus importantes sont:{str1_ax7}
    """
    ax7 += str2_ax7
    
worst_mtbf_cat = MTBF_MTTR_cat['MTBF en j'].idxmin()
worst_mtbf = int(MTBF_MTTR_cat['MTBF en j'].min())

ax7 += f'\nLe MTBF le moins bon du mois est celui de la catégorie "{worst_mtbf_cat}" :  {worst_mtbf} jours '


In [50]:
MTBF_MTTR_cat

,duree,freq,MTTR en h,MTBF en j
Catégorie,,,,
Brake,70.463333,7,10.066190,561.009147
Controller,176.647499,13,13.588269,301.741514
Converter-1,581.911945,14,41.565139,278.982405
Environement,76.186111,8,9.523264,490.853197
Gear,79.385555,16,4.961597,245.418267
Grid,3.186945,1,3.186945,3929.867211
Hub,79.852222,6,13.308704,654.445471
Hydraulics,77.174444,8,9.646805,490.848050
MISCELLANEOUS,14.612222,1,14.612222,3929.391157


In [37]:
# with open("Output.txt", "w", encoding='UTF-8') as text_file:
#     text_file.write(ax7)

## Graph 6 ax9

In [31]:
df_ax9 = pd.merge(
        (results.groupby('StationId')
                       .agg(**{'Durée arrêts': ('Duration 115(s)', lambda x: x.sum()/3600),
                               'Durée autres': ('Duration 20-25(s)', lambda x: x.sum()/3600)})
                       .sort_values('Durée arrêts', ascending=False)
                 # .head(25)
                 .reset_index()
         ),
         (alarms.groupby('StationId')
                      .agg(**{'Freq arrêts': ('Alarmcode', lambda x: int(x[x == 115].count()/2)),
                              'Freq autres': ('Alarmcode', lambda x: x[x == 20].count())})
                      .reset_index()
         ),
         on='StationId'
        ).sort_values('Durée arrêts', ascending=False).head(20)


df_ax9['Duration_20_25 reel'] = df_ax9['Durée autres']
df_ax9['Durée autres'] = (df_ax9['Duration_20_25 reel'] - df_ax9['Durée arrêts'])
# df_ax9.plot(kind='bar', x='StationId', ax=ax9)

df_ax9

,StationId,Durée arrêts,Durée autres,Freq arrêts,Freq autres,Duration_20_25 reel
0,75,720.000000,0.000000,0,0,720.000000
1,65,495.038611,36.241667,5,71,531.280278
2,114,183.990000,89.131667,21,45,273.121667
3,63,176.244722,36.957778,20,22,213.202500
4,32,131.282778,49.386667,38,51,180.669444
5,61,125.465000,57.488056,21,44,182.953056
6,45,121.145833,59.159444,15,37,180.305278
7,108,83.859722,93.831389,14,45,177.691111
8,121,79.109167,94.489167,16,44,173.598333
9,54,70.280833,73.614722,62,47,143.895556


## Graphe 7 ax10

In [32]:
df_ax10 = (results[['StationId','ELNX', 'EL_indefini_left']]
                .groupby('StationId')
                .sum()
                .sort_values('ELNX', ascending=False)
                .reset_index())

df_ax10[['ELNX', 'EL_indefini_left']] = df_ax10[['ELNX', 'EL_indefini_left']] / 1e3

df_ax10 = round(df_ax10, 2).abs().head(20)

df_ax10.rename({'ELNX': 'Energie perdue SGRE'}, inplace=True, axis=1)

df_ax10

,StationId,Energie perdue SGRE,EL_indefini_left
0,75,605.85,0.00
1,65,123.92,2.48
2,114,111.94,0.00
3,121,89.44,0.00
4,108,83.65,0.00
5,45,74.81,0.00
6,1,60.77,0.18
7,32,59.92,0.00
8,130,59.22,0.00
9,128,57.56,0.00


## ax11

In [33]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

period_month_name_ascii = remove_accents(period_month_name).upper()

In [35]:
parts_replaced = pd.read_excel(f'./input/{period}/PART REPLACED SGRE SERVICE {period_year}.xlsx', header=7, sheet_name=period_month_name_ascii)

df_ax11_last = pd.read_excel(f'./results/{previous_period}/TB Performance {previous_period}.xlsx', sheet_name='Feuil3', skiprows=1,
                       usecols='C:E')


parts_replaced = parts_replaced.rename(columns={'REFERENCE': 'item', 'QTE SORTIE': period_month_name})

parts_replaced['item'] = parts_replaced['item'].str.upper()

parts_replaced = parts_replaced.groupby('item').sum().reset_index()[['item', period_month_name]]



# if period_month == 1:

#     df_ax11 = parts_replaced
#     df_ax11['Year'] = df_ax11[period_month_name].fillna(0)
#     df_ax11 = df_ax11[['item', 'Year', period_month_name]]
    
# else:
df_ax11 = df_ax11_last.merge(parts_replaced, how='outer')[['item', 'Year', period_month_name]]
df_ax11['Year'] = (df_ax11['Year'].fillna(0) + df_ax11[period_month_name].fillna(0))

df_ax11 = df_ax11.fillna(0).astype(int, errors='ignore')

In [36]:
parts_replaced


,item,Avril
0,A9B00010003,1
1,A9B00012444,1
2,A9B00019620,1
3,A9B00019621,4
4,A9B00020977,3
5,A9B00020978,2
6,A9B00026953,1
7,A9B00056749,1
8,A9B00075055,2
9,A9B00081364,2


## ax12

In [38]:
df_ax12 = pd.read_excel(f'./input/{period}/PART REPLACED SGRE SERVICE {period_year}.xlsx', skiprows=7, sheet_name=period_month_name_ascii)

df_ax12[['DESIGNATION', 'TURBINE']] = df_ax12[['DESIGNATION', 'TURBINE']].apply(
    lambda s: s.str.upper().str.lstrip('\t'), 1)


df_ax12 = (df_ax12.groupby('DESIGNATION')
                  .agg({'QTE SORTIE': np.sum,
                        'TURBINE': lambda x: set(map(int, x.str.upper().str.lstrip('T0').str.cat(sep=' ').split()))})
                  .reset_index())


max_turbines = set(df_ax10.StationId.astype(int))
df_ax12['max_turbines'] = df_ax12['TURBINE'].apply(lambda x: x & max_turbines)

df_ax12.sort_values('QTE SORTIE').reset_index(drop=True).to_clipboard()

In [39]:
max_turbines

{1,
 3,
 32,
 43,
 44,
 45,
 50,
 51,
 54,
 61,
 63,
 65,
 75,
 108,
 110,
 114,
 118,
 121,
 128,
 130}

In [40]:
df_ax12 = pd.read_excel(f'./input/{period}/PART REPLACED SGRE SERVICE {period_year}.xlsx', skiprows=7, sheet_name=period_month_name_ascii)

df_ax12[['DESIGNATION', 'TURBINE']] = df_ax12[['DESIGNATION', 'TURBINE']].apply(
    lambda s: s.str.upper().str.lstrip('\t'), 1)

df_ax12 = df_ax12.groupby(['DESIGNATION', 'TURBINE']).sum().sort_values(['DESIGNATION', 'QTE SORTIE'])

df_ax12

QTE SORTIE
DESIGNATION                              TURBINE            
 COUPLING PUMP 38/45.25H7 CAST IRON      T33               1
ANEMOMETRE                               T91               1
                                         T1                2
BATTERY PACK (8 PC) FOR 809.C1 OR 809.C2 T109              1
                                         T110              1
...                                                      ...
YAW MOTOR                                T50               2
                                         T6                2
                                         T71               2
                                         T130              4
                                         T31               4

[111 rows x 1 columns]

In [42]:
# Turbines with no parts replaced

set(max_turbines) - set(df_ax12.reset_index()['TURBINE'].str.lstrip('T').astype(int))

{3, 63, 108, 114, 128}

## table 1 ax1 row 2

In [43]:
df_ax1 = pd.DataFrame(columns=['LTA-Lost Time', 'Indispo. Total %','Indispo. Tarec %', 'Indispo. Siemens %', 'Indispo. ONEE %', 'Indispo. Ebop %',
                               'Pertes élctriques en MWh', 'Power Boost en MWh', 'Performance moyenne des turbines',
                               'MTBF - Mean Time Between Failure', 'MTTR - Mean Time To Repair', 'MTTI - Mean Time To Intervention',
                               'Compteurs ONEE MWh'],
                     index=['Indicateurs annuels :', f'Indicateurs du mois {period} :'])
df_ax1 = df_ax1.fillna(0.)

In [44]:
df_ax1.iat[1, 1] = round(100*results['RealPeriod'].sum()/3600/24/131/days_in_period,2)
df_ax1.iat[1, 2] = round(100*results['Period 0(s)'].sum()/3600/24/131/days_in_period,2)
df_ax1.iat[1, 3] = round(100*results['Period 1(s)'].sum()/3600/24/131/days_in_period,2)

df_ax1.iat[1, 6] = round(results.wtc_kWG1TotE_accum.sum()/1000 - onee_period,2)

df_ax1.iat[1, 7] = boost_period

# df_ax1.iat[1, 8] = ratio_period
#
df_ax1.iat[1, 9]  = mtbf_period
df_ax1.iat[1, 10] = mttr_period
df_ax1.iat[1, 11] = mtti_period
df_ax1.iat[1, 12] = onee_period

In [45]:
round(results.wtc_kWG1TotE_accum.sum()/1000 - onee_period,2)


1571.49

In [46]:
df_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000
Indicateurs du mois 2021-04 :,0.0,4.0,1.26,2.74,0.0,0.0,1571.49,456.067602,0.0,43.550348,14.566926,4.321691,74786.624


## table 2 ax2

In [47]:

df_ax2_header = ['', '', '', 'Travaux de maintenance']
df_ax2 = pd.DataFrame(columns=['Transformateur', 'Serial Defect', 'Corrosion', 'Inspection Fin de garantie',
                               'FSA', 'Assurance', 'Conformité DNSSI', 'Main Bearing', 'BAX', 'Darwin',
                               'Procédure de securité', 'SMI', 'Exploitation', 'Mesures', 'Audit ENGIE'],
                     index=['Etape terminée', 'Etape en cours'])

In [48]:
df_ax2

,Transformateur,Serial Defect,Corrosion,Inspection Fin de garantie,FSA,Assurance,Conformité DNSSI,Main Bearing,BAX,Darwin,Procédure de securité,SMI,Exploitation,Mesures,Audit ENGIE
Etape terminée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Etape en cours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Calcul Cumul

In [49]:
cumul_alarms = pd.DataFrame()
cumul_results = pd.DataFrame()

for month in range(1, period_month+1):

    period_cumul = f'{str(period_year).zfill(2)}-{str(month).zfill(2)}'
    alarms = read_sum(f'{period_cumul}')

    cumul_alarms = pd.concat([cumul_alarms, alarms])

    # -------------------------------------------------------------------------
    results = pd.read_pickle(f'../DATA/results/{period_cumul}.pkl')
    
    # results = results[['StationId', 'ELNX', 'Duration 115(s)', 'Duration 20-25(s)',
    #                    'Period 0(s)', 'Period 1(s)', 'RealPeriod',
    #                    'EL_indefini_left']]

    cumul_results = pd.concat([cumul_results, results])


DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-01-sum.mdb;
../DATA/SUM/2021-01-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-02-sum.mdb;
../DATA/SUM/2021-02-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-03-sum.mdb;
../DATA/SUM/2021-03-sum.mdb Loaded
DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2021-04-sum.mdb;
../DATA/SUM/2021-04-sum.mdb Loaded


In [50]:
cumul_alarms.rename(columns={'StationNr': 'StationId'}, inplace=True)

# only run once otherwise negative turbine numbers
cumul_alarms['StationId'] = cumul_alarms['StationId'] - 2307404
cumul_results['StationId'] = cumul_results['StationId'] - 2307404
results['StationId'] = results['StationId'] - 2307404

In [51]:
cumul_result_sum = pd.merge(cumul_alarms, error_list[[
                        'Alarmcode', 'Error Type', 'Error Group']],
                        on='Alarmcode',
                        how='inner', sort=False)

# Remove warnings
cumul_result_sum = cumul_result_sum.loc[cumul_result_sum['Error Type'].isin([1, 0])]

# apply cascade
cumul_alarms_result_sum = apply_cascade(cumul_result_sum)

#only keep  parent alarms
cumul_parent_result_sum = cumul_alarms_result_sum.query('TimeOn	 == NewTimeOn')
  
cumul_main_result_sum = cumul_alarms_result_sum.query('RealPeriod > @pd.Timedelta(0)')

## MTTI MTTR MTBF cumulé

In [52]:

mtbf_cumul = (last_mtbf_cumul*days_to_previous_period + mtbf_period *
              days_in_period)/(days_to_previous_period+days_in_period)

mttr_cumul = (last_mttr_cumul*days_to_previous_period + mttr_period *
              days_in_period)/(days_to_previous_period+days_in_period)

mtti_cumul = (last_mtti_cumul*days_to_previous_period + mtti_period *
              days_in_period)/(days_to_previous_period+days_in_period)

## Table 2 ax1 row 1

In [53]:
nbr_jrs_total = len(pd.date_range(f'{period_year}-01-01', f'{lastday_period}', freq='D'))

df_ax1.iat[0, 1] = round(100*cumul_results['RealPeriod'].sum()/3600/24/131/nbr_jrs_total,2)
df_ax1.iat[0, 2] = round(100*cumul_results['Period 0(s)'].sum()/3600/24/131/nbr_jrs_total,2)
df_ax1.iat[0, 3] = round(100*cumul_results['Period 1(s)'].sum()/3600/24/131/nbr_jrs_total,2)

df_ax1.iat[0, 6] = round(cumul_results.wtc_kWG1TotE_accum.sum()/1000 - onee_cumul,2)

df_ax1.iat[0, 7] = boost_cumul

# df_ax1.iat[0, 8] = ratio_cumul
# 
df_ax1.iat[0, 9] =  mtbf_cumul
df_ax1.iat[0, 10] = mttr_cumul
df_ax1.iat[0, 11] = mtti_cumul
df_ax1.iat[0, 12] = onee_cumul

In [54]:
100*round(cumul_results.wtc_kWG1TotE_accum.sum()/1000 - onee_cumul,2)/(cumul_results.wtc_kWG1TotE_accum.sum()/1000)

2.0536886056725145

In [55]:
df_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0.0,4.21,0.83,3.38,0.0,0.0,6657.64,2169.643189,0.0,28.742819,10.277235,6.394152,317521.985
Indicateurs du mois 2021-04 :,0.0,4.00,1.26,2.74,0.0,0.0,1571.49,456.067602,0.0,43.550348,14.566926,4.321691,74786.624


### Yassine Hakim

In [60]:
nbr_jrs_total = len(pd.date_range(f'{period_year}-01-01', f'{lastday_period}', freq='D'))


(round(100*results['Period 0(s)'].sum()/3600/24/131/days_in_period,2),
round(100*results['Period 1(s)'].sum()/3600/24/131/days_in_period,2),
round(100*results['RealPeriod'].sum()/3600/24/131/days_in_period,2),
round(results.wtc_kWG1TotE_accum.sum()/1000 - onee_period,2),
round(results[['met_WindSpeedRot_mean_38', 'met_WindSpeedRot_mean_39', 'met_WindSpeedRot_mean_246']].mean().mean(), 2))


(1.26, 2.74, 4.0, 1571.49, 7.34)

## Graphe 1 ax3

In [61]:
df_ax3 = (cumul_main_result_sum.groupby('Error Group')
                .agg(Freq=('Alarmcode', 'count'),
                     Durée=('RealPeriod', lambda x: x.sum().total_seconds()/3600))
                .sort_values('Freq', ascending=False)
                .reindex(reorder)
                .dropna()
                .reset_index()
                )

# df_ax3.plot(kind='bar', x='Error Group', ax=ax3)
df_ax3

,Error Group,Freq,Durée
0,System,2202.0,7753.355202
1,Generator,13.0,7.673333
2,Hub,3162.0,875.194990
3,Gear,183.0,578.099999
4,Grid,136.0,73.985551
5,Rotor,24.0,17.839444
6,Hydraulics,1334.0,1303.703056
7,Environement,344.0,324.024445
8,Brake,706.0,387.664967
9,Yaw,3152.0,2007.123361


## Graphe 2 ax4

In [62]:
MTBF_MTTR_cat = (main_result_sum.query(query).groupby('Error Group')
                                .agg(duree=('RealPeriod', lambda x: x.sum().total_seconds() / 3600),
                                     freq=('TimeOn', 'count'),
                                     )
                                .rename_axis('Catégorie'))

In [63]:
MTBF_MTTR_cat

,duree,freq
Catégorie,,
Brake,70.463333,7
Controller,176.647499,13
Converter-1,581.911945,14
Environement,76.186111,8
Gear,79.385555,16
Grid,3.186945,1
Hub,79.852222,6
Hydraulics,77.174444,8
MISCELLANEOUS,14.612222,1


In [64]:
MTBF_MTTR_cat_prev_cumul = pd.read_excel(
    f'../TB Performance/input/{previous_period}/output_xlsxwriter.xlsx', sheet_name='MTBF & MTTR cumul')

MTBF_MTTR_cat_prev_cumul.dropna(
    how='all', subset=['MTBF en j', 'MTTR en h'], inplace=True)

MTBF_MTTR_cat_prev_cumul.rename(
    columns={'Catégorie Alarme': 'Catégorie'}, inplace=True)

MTBF_MTTR_cat_prev_cumul.set_index('Catégorie', inplace=True)

MTBF_MTTR_cat_prev_cumul

,duree,freq,MTTR en h,MTBF en j
Catégorie,,,,
Brake,941.188744,100,9.411887,117.507838
Controller,3141.018728,261,12.034555,44.670974
Converter,2790.053182,268,10.410646,43.558760
Environement,353.985833,45,7.866352,261.672235
Gear,1399.334165,168,8.329370,69.831514
Grid,2736.168617,264,10.364275,44.227246
Hub,1613.669720,145,11.128757,80.846647
Hydraulics,3691.181113,459,8.041789,25.351200
MISCELLANEOUS,277.467778,25,11.098711,471.137554


In [65]:
MTBF_MTTR_cat_cumul = MTBF_MTTR_cat.merge(MTBF_MTTR_cat_prev_cumul, on='Catégorie', how='outer').fillna(0)

MTBF_MTTR_cat_cumul.loc['Converter-1'] = MTBF_MTTR_cat_cumul.loc[['Converter-1', 'Converter']].sum()

MTBF_MTTR_cat_cumul = (MTBF_MTTR_cat_cumul.drop('Converter')
                                          .rename(index={'Converter-1': 'Converter'}))

MTBF_MTTR_cat_cumul

,duree_x,freq_x,duree_y,freq_y,MTTR en h,MTBF en j
Catégorie,,,,,,
Brake,70.463333,7.0,941.188744,100.0,9.411887,117.507838
Controller,176.647499,13.0,3141.018728,261.0,12.034555,44.670974
Converter,581.911945,14.0,2790.053182,268.0,10.410646,43.558760
Environement,76.186111,8.0,353.985833,45.0,7.866352,261.672235
Gear,79.385555,16.0,1399.334165,168.0,8.329370,69.831514
Grid,3.186945,1.0,2736.168617,264.0,10.364275,44.227246
Hub,79.852222,6.0,1613.669720,145.0,11.128757,80.846647
Hydraulics,77.174444,8.0,3691.181113,459.0,8.041789,25.351200
MISCELLANEOUS,14.612222,1.0,277.467778,25.0,11.098711,471.137554


In [66]:
# Generate the new calculated MTTBF and MTTR for each error group

temps_total_cumul = len(pd.date_range(
    f'{period_year}-01-01', f'{lastday_period}', freq='D'))*24*131

MTBF_MTTR_cat_cumul['duree_y'] = MTBF_MTTR_cat_cumul['duree_x'] + MTBF_MTTR_cat_cumul['duree_y']

duree_cumul = MTBF_MTTR_cat_cumul['duree_y']

MTBF_MTTR_cat_cumul['freq_y'] = MTBF_MTTR_cat_cumul['freq_x'] + MTBF_MTTR_cat_cumul['freq_y']

freq_cumul = MTBF_MTTR_cat_cumul['freq_y']

MTBF_MTTR_cat_cumul['MTTR en h'] = duree_cumul/freq_cumul

MTBF_MTTR_cat_cumul['MTBF en j'] = (
    temps_total_cumul - duree_cumul) / (freq_cumul * 24)


In [67]:
MTBF_MTTR_cat_cumul = MTBF_MTTR_cat_cumul[['duree_y', 'freq_y', 'MTTR en h', 'MTBF en j']].reset_index(
).rename(columns={'duree_y': 'duree', 'freq_y': 'freq'})


In [68]:
MTBF_MTTR_cat_cumul

,Catégorie,duree,freq,MTTR en h,MTBF en j
0,Brake,1011.652077,107.0,9.454692,146.521942
1,Controller,3317.666228,274.0,12.108271,56.867751
2,Converter,3371.965127,282.0,11.957323,55.246459
3,Environement,430.171944,53.0,8.116452,296.265588
4,Gear,1478.719720,184.0,8.036520,85.099928
5,Grid,2739.355562,265.0,10.337191,58.890038
6,Hub,1693.521942,151.0,11.215377,103.638653
7,Hydraulics,3768.355557,467.0,8.069284,33.325450
8,MISCELLANEOUS,292.080001,26.0,11.233846,604.147308
9,Yaw,5824.849436,530.0,10.990282,29.202449


In [69]:
df_ax4 = MTBF_MTTR_cat_cumul[['Catégorie', 'MTTR en h', 'MTBF en j']]
# df_ax4.rename(columns={'MTTR en h': 'MTTR', 'MTBF en j': 'MTBF'}, inplace=True)

In [70]:
df_ax4

,Catégorie,MTTR en h,MTBF en j
0,Brake,9.454692,146.521942
1,Controller,12.108271,56.867751
2,Converter,11.957323,55.246459
3,Environement,8.116452,296.265588
4,Gear,8.036520,85.099928
5,Grid,10.337191,58.890038
6,Hub,11.215377,103.638653
7,Hydraulics,8.069284,33.325450
8,MISCELLANEOUS,11.233846,604.147308
9,Yaw,10.990282,29.202449


## Graphe 5 ax8

In [71]:
df_ax8 = pd.merge(
        (cumul_results.groupby('StationId')
                       .agg(**{'Durée arrêts': ('Duration 115(s)', lambda x: x.sum()/3600),
                               'Durée autres': ('Duration 20-25(s)', lambda x: x.sum()/3600)})
                       .reset_index()
         ),
         (cumul_alarms.groupby('StationId')
                      .agg(**{'Freq arrêts': ('Alarmcode', lambda x: int(x[x == 115].count()/2)),
                              'Freq autres': ('Alarmcode', lambda x: x[x == 20].count())})
                      .reset_index()
         ),
        ).sort_values('Durée arrêts', ascending=False).head(20)

# df_ax8.plot(kind='bar', x='StationId', ax=ax8)

# for stacked bar excel chart
df_ax8['Duration_20_25 reel'] = df_ax8['Durée autres']
df_ax8['Durée autres'] = df_ax8['Duration_20_25 reel'] - df_ax8['Durée arrêts']

df_ax8

,StationId,Durée arrêts,Durée autres,Freq arrêts,Freq autres,Duration_20_25 reel
74,75,1673.453889,163.601111,33,79,1837.055000
59,60,1479.646389,129.458333,51,105,1609.104722
64,65,784.414167,175.607222,89,275,960.021389
100,101,714.984722,276.941944,34,118,991.926667
63,64,674.216111,212.351111,50,139,886.567222
113,114,402.407778,323.283056,87,193,725.690833
53,54,306.131389,265.561111,224,180,571.692500
101,102,277.630833,207.121389,101,501,484.752222
44,45,274.059444,278.453889,130,226,552.513333
103,104,258.045556,314.428611,53,184,572.474167


## str ax12

In [72]:
Ep_cumul = cumul_results['wtc_kWG1TotE_accum'].sum()
EL_cumul = cumul_results['EL'].sum()
ELX_cumul = cumul_results['ELX'].sum()
ELNX_cumul = cumul_results['ELNX'].sum()
EL_2006_cumul = cumul_results['EL_2006'].sum()
Epot_cumul = cumul_results['Epot'].sum()

EL_wind_cumul = cumul_results['EL_wind'].sum()
EL_wind_start_cumul = cumul_results['EL_wind_start'].sum()
EL_alarm_start_cumul = cumul_results['EL_alarm_start'].sum()

MAA_cumul = round(100 * (
    Ep_cumul + ELX_cumul) / (
        Ep_cumul + EL_cumul - (EL_wind_cumul + EL_wind_start_cumul + EL_alarm_start_cumul + EL_2006_cumul)), 2)


MAA_cumul_brut = round(100 * (
    Ep_cumul + ELX_cumul) / (
        Ep_cumul + ELX_cumul + ELNX_cumul + EL_2006_cumul), 2)


In [73]:
MAA_cumul, MAA_cumul_brut

(96.16, 96.19)

In [83]:
(round(100 * ELX_cumul / (Ep_cumul + ELX_cumul + ELNX_cumul + EL_2006_cumul), 2),
 round(100 * (ELNX_cumul + EL_2006_cumul) / (Ep_cumul + ELX_cumul + ELNX_cumul + EL_2006_cumul), 2))


(0.48, 3.81)

In [79]:
Ep_period = results['wtc_kWG1TotE_accum'].sum()
EL_period = results['EL'].sum()
ELX_period = results['ELX'].sum()
ELNX_period = results['ELNX'].sum()
EL_2006_period = results['EL_2006'].sum()

Epot_period = results['Epot'].sum()

EL_wind_period = results['EL_wind'].sum()
EL_wind_start_period = results['EL_wind_start'].sum()
EL_alarm_start_period = results['EL_alarm_start'].sum()

MAA_period = round(100 * (
    Ep_period + ELX_period) / (
        Ep_period + EL_period - (EL_wind_period + EL_wind_start_period + EL_alarm_start_period + EL_2006_period)), 2)

MAA_period_brut = round(100 * (
    Ep_period + ELX_period) / (
        Ep_period + ELX_period + ELNX_period + EL_2006_period), 2)
        

In [28]:
MAA_period, MAA_period_brut

(97.11, 97.19)

In [82]:
(round(100 * ELX_period / (Ep_period + ELX_period + ELNX_period + EL_2006_period), 2),
 round(100 * (ELNX_period + EL_2006_period) / (Ep_period + ELX_period + ELNX_period + EL_2006_period), 2))


(0.86, 2.81)

In [90]:
ELNX_cumul = round((ELNX_cumul + EL_2006_cumul) / 1e6, 2)
ELNX_period = round((ELNX_period + EL_2006_period) / 1e6, 2)


In [84]:
# str_ax12 = f"""ELNX imputée à SGRE:  {ELNX_cumul} GWh ({ELNX_period} GWh pour {period_month_name} {period_year})
# Disponibilité en énergie selon FSA : {MAA_cumul_brut}% ({MAA_period_brut} % pour {period_month_name} {period_year})
# Disponibilité en énergie constatée : {MAA_cumul}% ({MAA_period} % pour {period_month_name} {period_year})"""

In [91]:
str_ax12 = f"""Energie imputée à SGRE: {period_year} YTD {ELNX_cumul} GWh & Mensuel {ELNX_period} GWh
Disponibilité en énergie selon alarmes FSA: {period_year} YTD {MAA_cumul_brut} % & Mensuel {MAA_period_brut} %"""


In [92]:
str_ax12

'Energie imputée à SGRE: 2021 YTD 12.9 GWh & Mensuel 2.23 GWh\nDisponibilité en énergie selon alarmes FSA: 2021 YTD 96.19 % & Mensuel 97.19 %'

## Graph 7 ax18

In [93]:
df_ax18 = (cumul_results[['StationId','ELNX', 'EL_indefini_left']]
                .groupby('StationId')
                .sum()
                .sort_values('StationId')
                # .head(20)
                .reset_index())

df_ax18[['ELNX', 'EL_indefini_left']] = df_ax18[['ELNX', 'EL_indefini_left']] / 1e3

df_ax18.rename({'ELNX': 'Energie perdue SGRE'}, inplace=True, axis=1)

df_ax18 = df_ax18.round(2)

In [94]:
df_ax18

,StationId,Energie perdue SGRE,EL_indefini_left
0,1,79.83,0.18
1,2,19.74,0.07
2,3,103.93,-0.00
3,4,47.67,-0.00
4,5,135.23,0.96
...,...,...,...
126,127,81.44,0.17
127,128,117.04,1.86
128,129,11.96,0.02
129,130,61.56,0.13


## ax19

In [95]:
from scipy.stats import binned_statistic
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy import integrate
from scipy.interpolate import interp1d


In [96]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [97]:
CB2 = pd.read_excel('CB2.xlsx')

CB2 = CB2.astype(int).drop_duplicates()

CB2_interp = interp1d(CB2.Wind,
              CB2.Power, kind='linear', 
              fill_value="extrapolate")

In [98]:
# import warnings
# warnings.filterwarnings("error")

def ratio_cp(df, CB2_interp, name):

    df = df.query('RealPeriod == 0')
    
    if len(df) == 0:
        return np.nan
    
    x = df.wtc_AcWindSp_mean.values
    y = df.wtc_ActPower_mean.values.clip(0, 2300)
    
    bin_means, bin_edges, binnumber = (
        binned_statistic(x, y, statistic='mean', bins=25))
    
    bins = running_mean(bin_edges, 2)

    nan_index_means = np.argwhere(
        np.isnan(
            bin_means
        )
    )

    bin_means = np.delete(bin_means, nan_index_means)

    bins = np.delete(bins, nan_index_means)
    
    df_interp = interp1d(bins, bin_means, kind='linear', 
                         fill_value="extrapolate")
    
    nan_index = np.argwhere(
        np.isnan(
            df_interp(bins)
        )
    )

    bins = np.delete(bins, nan_index)
    
    try:
    
        integral_df = quad(df_interp, bins.min(), bins.max(),
                           points=bins,
                           limit=100)
    
        integral_siemens = quad(CB2_interp,
                                bins.min(),bins.max(),
                                points=CB2.Wind, 
                                limit=100)

    except: 
        print('except tur:')
        print(name)
        print(len(df))
        return None
    
    # print('tur')
    # print(name)

    # return 100 * integral_df[0] / integral_siemens[0]

    return 100 * (y.sum() / CB2_interp(x).sum())

    # return (100 * (df.wtc_ActPower_mean / CB2_interp(x))).replace(np.NINF, 100).mean()


    

In [99]:
df_ax19 = (results.groupby('StationId')
                  .apply(lambda df: ratio_cp(df, CB2_interp, df.name))
                  .to_frame('Ratio')
                  .reset_index())

In [100]:
df_ax19.Ratio.mean()

100.48294462991167

In [101]:
df_ax19.iloc[df_ax19.Ratio.idxmin()]

StationId    122.000000
Ratio         83.934051
Name: 121, dtype: float64

In [102]:
df_ax19

,StationId,Ratio
0,1,96.017650
1,2,104.111445
2,3,95.821308
3,4,95.766618
4,5,87.125493
...,...,...
126,127,95.644137
127,128,110.368767
128,129,92.739941
129,130,95.733096


## ratio period et cumul

In [103]:
ratio_period = df_ax19.Ratio.mean()
df_ax1.iat[1, 8] = ratio_period

In [104]:
ratio_cumul = ((last_ratio_cumul*days_to_previous_period + ratio_period*days_in_period)
               /(days_to_previous_period+days_in_period))

df_ax1.iat[0, 8] = ratio_cumul

In [105]:
ratio_cumul, ratio_period


(99.96578714867692, 100.48294462991167)

# Export XLSX writer

In [107]:
writer = pd.ExcelWriter(f'./input/{period}/output_xlsxwriter.xlsx')
# -------------------------------------------------------------------------------------------

workbook = writer.book
dashsheet = workbook.add_worksheet('Dash')

# -------------------------------------------------------------------------------------------

MTBF_MTTR_cat_cumul.to_excel(writer, sheet_name='MTBF & MTTR cumul', index=False)

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax1-------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax1.to_excel(writer, sheet_name='ax1')

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax3-------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax3.to_excel(writer, index=False, sheet_name='ax3')

worksheet = writer.sheets['ax3']


def make_chart_ax3():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax3', 0, 2],
                             'categories': ['ax3', 1, 0, 13, 0],
                             'values': ['ax3', 1, 2, 13, 2], })

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax3', 1, 1, 13, 1],
        'categories': ['ax3', 1, 0, 13, 0],
        'name': ['ax3', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax3.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'Cumul annuel par type d\'alarme',
                            'name_font': {'size': 12, 'bold': True}})
    line_chart.set_y2_axis({'name': df_ax3.columns[1]})
    return column_chart


column_chart = make_chart_ax3()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax3()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B2', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax4-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax4.to_excel(writer, index=False, sheet_name='ax4')
worksheet = writer.sheets['ax4']


def make_chart_ax4():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax4', 0, 2],
                             'categories': ['ax4', 1, 0, 13, 0],
                             'values': ['ax4', 1, 2, 13, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax4', 1, 1, 13, 1],
        'categories': ['ax4', 1, 0, 13, 0],
        'name': ['ax4', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)
    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax4.columns[0]})
    column_chart.set_y_axis({'name': 'MTBF en j'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'MTBF et MTTR par catégorie sur l\'année 2021',
                            'name_font': {'size': 12, 'bold': True}})
    line_chart.set_y2_axis({'name': 'MTTR en h'})
    return column_chart


column_chart = make_chart_ax4()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax4()
# Insert the chart into the worksheet.
dashsheet.insert_chart('J2', column_chart)



# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax5-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax5.to_excel(writer, index=False, sheet_name='ax5')

worksheet = writer.sheets['ax5']


def make_chart_ax5():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax5', 0, 2],
                             'categories': ['ax5', 1, 0, 13, 0],
                             'values': ['ax5', 1, 2, 13, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax5', 1, 1, 13, 1],
        'categories': ['ax5', 1, 0, 13, 0],
        'name': ['ax5', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax5.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Type d\'alarme {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': df_ax5.columns[1]})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax5()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax5()
# Insert the chart into the worksheet.
dashsheet.insert_chart('R2', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax6-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax6.to_excel(writer, index=False, sheet_name='ax6')

worksheet = writer.sheets['ax6']


def make_chart_ax6():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax6', 0, 2],
                             'categories': ['ax6', 1, 0, 20, 0],
                             'values': ['ax6', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax6', 1, 1, 20, 1],
        'categories': ['ax6', 1, 0, 20, 0],
        'name': ['ax6', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax6.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Alarmes {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': df_ax6.columns[1]})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax6()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax6()
# Insert the chart into the worksheet.
dashsheet.insert_chart('Z2', column_chart)

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax7-------------------------------------------------
# -------------------------------------------------------------------------------------------

worksheet = workbook.add_worksheet('ax7')

options = {
    'x_scale': 3,
    'y_scale': 25,
}

worksheet.insert_textbox(0, 0, ax7, options)



# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax8-------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax8.to_excel(writer, index=False, sheet_name='ax8')

worksheet = writer.sheets['ax8']


def make_chart_ax8():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    column_chart.add_series({'name': ['ax8', 0, 1],
                             'categories': ['ax8', 1, 0, 20, 0],
                             'values': ['ax8', 1, 1, 20, 1]})

    column_chart.add_series({'name': ['ax8', 0, 2],
                             'categories': ['ax8', 1, 0, 20, 0],
                             'values': ['ax8', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax8', 1, 3, 20, 3],
        'categories': ['ax8', 1, 0, 20, 0],
        'name': ['ax8', 0, 3],
        'y2_axis': True,
    })

    line_chart.add_series({
        'values': ['ax8', 1, 4, 20, 4],
        'categories': ['ax8', 1, 0, 20, 0],
        'name': ['ax8', 0, 4],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax8.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'Arrêts turbines : Cumul Annuel',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': 'Freq'})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax8()
# Insert the chart into the worksheet.
worksheet.insert_chart('G2', column_chart)

column_chart = make_chart_ax8()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B17', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax9-------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax9.to_excel(writer, index=False, sheet_name='ax9')

worksheet = writer.sheets['ax9']


def make_chart_ax9():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    column_chart.add_series({'name': ['ax9', 0, 1],
                             'categories': ['ax9', 1, 0, 20, 0],
                             'values': ['ax9', 1, 1, 20, 1]})

    column_chart.add_series({'name': ['ax9', 0, 2],
                             'categories': ['ax9', 1, 0, 20, 0],
                             'values': ['ax9', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax9', 1, 3, 20, 3],
        'categories': ['ax9', 1, 0, 20, 0],
        'name': ['ax9', 0, 3],
        'y2_axis': True,
    })

    line_chart.add_series({
        'values': ['ax9', 1, 4, 20, 4],
        'categories': ['ax9', 1, 0, 20, 0],
        'name': ['ax9', 0, 4],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax9.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Arrêts turbines {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': 'Freq'})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax9()
# Insert the chart into the worksheet.
worksheet.insert_chart('G2', column_chart)

column_chart = make_chart_ax9()
# Insert the chart into the worksheet.
dashsheet.insert_chart('J17', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax10------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax10.to_excel(writer, index=False, sheet_name='ax10')

worksheet = writer.sheets['ax10']


def make_chart_ax10():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    for col in range(1, 3):
        column_chart.add_series({'values': ['ax10', 1, col, 20, col],
                                 'categories': ['ax10', 1, 0, 20, 0],
                                 'name': ['ax10', 0, col]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax10.columns[0]})
    # column_chart.set_y_axis({'name': df_ax10.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': f'Energie perdue selon FSA du {period}',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax10()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax10()
# Insert the chart into the worksheet.
dashsheet.insert_chart('R17', column_chart)


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax11------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax11.to_excel(writer, index=False, sheet_name='ax11')


# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax12------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax12.to_excel(writer, sheet_name='ax12')

worksheet = writer.sheets['ax12']

options = {
    'x_scale': 4,
    'y_scale': 1,
}

worksheet.insert_textbox(0, 6, str_ax12, options)

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax18------------------------------------------------
# -------------------------------------------------------------------------------------------

df_ax18.to_excel(writer, index=False, sheet_name='ax18')

worksheet = writer.sheets['ax18']


def make_chart_ax18():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    for col in range(1, 3):
        column_chart.add_series({'values': ['ax18', 1, col, 131, col],
                                 'categories': ['ax18', 1, 0, 131, 0],
                                 'name': ['ax18', 0, col]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax18.columns[0]})
    # column_chart.set_y_axis({'name': df_ax18.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': 'Energie perdue selon FSA cumulée sur l\'année 2020 en MWh',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax18()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax18()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B32', column_chart, {'x_scale': 4, 'y_scale': 1})

# -------------------------------------------------------------------------------------------
# ------------------------------------df_ax19------------------------------------------------
# -------------------------------------------------------------------------------------------
df_ax19.to_excel(writer, index=False, sheet_name='ax19')

worksheet = writer.sheets['ax19']


def make_chart_ax19():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'values': ['ax19', 1, 1, 131, 1],
                             'categories': ['ax19', 1, 0, 131, 0],
                             'name': ['ax19', 0, 1]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax19.columns[0]})
    # column_chart.set_y_axis({'name': df_ax19.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': f'Ratio Courbe Théorique vs Courbe Réelle du {period}',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax19()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax19()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B47', column_chart, {'x_scale': 4, 'y_scale': 1})


writer.save()

In [ ]:
# !jupyter nbconvert performance.ipynb --to markdown